In [5]:
# data
import os

from nltk.corpus import stopwords

path_root = "C:/BCSpace/study/ThisSemester/IR/lab/lab5"
dir_source = "20_newsgroups"
dir_temp = "_temp"  # folder name for the processed files
file_indexer = 'indexer.csv'    # new file to store the indexer

path_source = os.path.join(path_root, dir_source)
path_temp = os.path.join(path_root, dir_temp)
doc_id = {}
doc_list = []
doc_len = {}

stop_words = {'i', 'me', 'my', 'myself', 'we', 
              'our', 'ours', 'ourselves', 'you', 
              "you're", "you've", "you'll", "you'd", 
              'your', 'yours', 'yourself', 'yourselves', 
              'he', 'him', 'his', 'himself', 'she', 
              "she's", 'her', 'hers', 'herself', 'it', 
              "it's", 'its', 'itself', 'they', 'them', 
              'their', 'theirs', 'themselves', 'what', 
              'which', 'who', 'whom', 'this', 'that', 
              "that'll", 'these', 'those', 'am', 'is', 
              'are', 'was', 'were', 'be', 'been', 
              'being', 'have', 'has', 'had', 'having', 
              'do', 'does', 'did', 'doing', 'a', 'an', 
              'the', 'and', 'but', 'if', 'or', 'because', 
              'as', 'until', 'while', 'of', 'at', 'by', 
              'for', 'with', 'about', 'against', 'between', 
              'into', 'through', 'during', 'before', 'after', 
              'above', 'below', 'to', 'from', 'up', 'down', 
              'in', 'out', 'on', 'off', 'over', 'under', 
              'again', 'further', 'then', 'once', 'here', 
              'there', 'when', 'where', 'why', 'how', 
              'all', 'any', 'both', 'each', 'few',
              'more', 'most', 'other', 'some', 'such', 
              'no', 'nor', 'not', 'only', 'own', 
              'same', 'so', 'than', 'too', 'very', 
              's', 't', 'can', 'will', 'just', 'don', 
              "don't", 'should', "should've", 'now', 'd', 
              'll', 'm', 'o', 're', 've', 'y', 'ain', 
              'aren', "aren't", 'couldn', "couldn't", 
              'didn', "didn't", 'doesn', "doesn't", 'hadn', 
              "hadn't", 'hasn', "hasn't", 'haven', "haven't", 
              'isn', "isn't", 'ma', 'mightn', "mightn't", 
              'mustn', "mustn't", 'needn', "needn't", 'shan', 
              "shan't", 'shouldn', "shouldn't", 'wasn', 
              "wasn't", 'weren', "weren't", 'won', "won't", 
              'wouldn', "wouldn't"}
# stop_words = set(stopwords.words('english'))
stop_words.add('s')


In [6]:
# Some functions
import csv
from math import log10

from nltk import RegexpTokenizer
import sys


# Recursively finds size of objects
def get_size(obj, seen=None):
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size


# tokenize

def get_term_freq(text):
    text = text.lower()
    # word_tokenizer = RegexpTokenizer(r'\w+|[^\w\s]')
    word_tokenizer = RegexpTokenizer('[A-Za-z]+')
    words = word_tokenizer.tokenize(text)
    term_set = set(words).difference(stop_words)
    term_freq = {}
    for w in words:
        if w in term_set:
            if w in term_freq:
                term_freq[w] = term_freq[w] + 1
            else:
                term_freq[w] = 1
    return term_freq, len(words)


# store sub dictionary in .csv

In [7]:
# Calculate the indexer and store into 'indexer.csv'

# get the sub dictionaries
from sys import getsizeof
from math import sqrt

chunk_size = 15000000


def process_directions():
    chunk_num = 0
    term_index = {}
    doc_list.clear()
    doc_id.clear()
    doc_len.clear()
    if not os.path.exists(path_temp):
        os.mkdir(path_temp)
    for root, dirs, files in os.walk(path_source):
        for file_name in files:
            fpn = os.path.join(root, file_name)
            doc_list.append(fpn)
            file_cnt = len(doc_list) - 1
            doc_id[fpn] = file_cnt

            with open(fpn, mode='r', errors='ignore', encoding='utf-8') as f:
                text = f.read()
            term_freq, d_len = get_term_freq(text)
            doc_len[file_cnt] = d_len
            for term, value in term_freq.items():
                if term in term_index:
                    term_index[term].append((file_cnt, value))
                else:
                    term_index[term] = [(file_cnt, value)]
    return term_index

indexer = process_directions()

KeyboardInterrupt: 

In [17]:
from math import log

def weighted_tf(tf, d_len, avg_len):
    return 2 * tf / (0.25 + 0.75 * d_len / avg_len + tf)

def idf(df, n):
    if df == 0:
        return 0
    return log(n / df, 10)

def hw3_bm25_11812420(query, doc_len = {}, dictionary = {}):
    avg_doc_len = 0
    doc_num = len(doc_len)
    for dl in doc_len:
        avg_doc_len = avg_doc_len + dl / doc_num

    candidate = {}
    for k in query:
        df = len(dictionary[k])
        for did, tf in dictionary[k]:
            if did not in candidate.keys():
                candidate[did] = 0
            candidate[did] = candidate[did] + weighted_tf(tf, doc_len[did], avg_doc_len) * idf(df, doc_num)

    candidate = sorted(candidate.items(), key= lambda x: (x[1]), reverse= True)
    return candidate[:10]

# Query
# for v in indexer['russia']:
#     print(doc_list[v[0]][len(path_root):], v[1])
query = ['china', 'japan', 'russia']
#
answer = hw3_bm25_11812420(query, doc_len, indexer)
print(answer)






china: 94 265 274 474 746 749 816 829 976 1659 3090 3711 3712 3713 3714 3896 5319 6410 8219 10120 11430 12730 12965 13458 13567 13645 13726 14012 14042 14556 15295 15446 16009 16010 16045 16177 16628 16766 16788 17007 17008 17036 17237 17444 17446 17502 17656 17677 17716 17750 17902 18004 18016 18025 18026 18087 18145 18209 18313 18371 18388 18437 19019 19109 19171 19232 19568 19629 19666 
japan: 443 474 487 1129 1138 1215 1216 1520 1647 1689 1691 1896 1925 1942 1958 2169 2820 3407 3417 4216 4241 4252 4311 5035 5319 5482 5561 5783 5897 6335 6411 6414 6443 7032 7075 7162 7456 7466 7488 7763 7767 7770 7858 7862 7879 7952 8074 8269 8492 8865 8892 8933 9173 9208 9259 9269 9275 9544 9752 9779 9870 9951 10080 10120 11030 11348 11443 11450 11654 11671 11704 11714 13186 13220 13645 13702 14005 14011 14012 14014 14015 14017 14022 14071 14072 14106 14172 14175 14185 14228 14230 14492 14504 14556 14602 14654 14665 14675 14978 15514 15555 15772 16316 16692 16781 16841 17178 17219 17324 17415 17438